In [1]:
import pandas as pd

In [2]:
file_path ='itmo_sber.csv'
data = pd.read_csv('itmo_sber.csv')

In [3]:
print(data.isnull().sum())

Unnamed: 0                    0
Задача                        0
Обстановка                    0
Оптимальный план              0
Задача en                     0
Обстановка en                 0
Оптимальный план en           0
Предсказанный план            0
Успех предсказанного плана    0
dtype: int64


In [4]:
print(data.head())

   Unnamed: 0                          Задача  \
0           0   Приготовить горячий шоколад.    
1           1     Заказать доставку еды домой   
2           2  Реставрация старого комода\r\n   
3           3            Приготовить сырники.   
4           4               Нарезать колбасу.   

                                          Обстановка  \
0  кухня, молоко, какао, сахар, кастрюля, ложка, ...   
1  мобильный телефон, ресторанное меню, интернет,...   
2  Старый комод, повреждения, шпатель, шкурка, ма...   
3  Кухня, столешница, творог, мука, яйцо, сахар, ...   
4  Кухня, колбаса, нож, доска для резки, холодиль...   

                                    Оптимальный план  \
0  Откройте дверь. Зайдите на кухню. Закрйоте две...   
1  Найдите стол. Подойдите к столу. Найдите мобил...   
2  Подойдите к комоду.Найдите шкурку.Возьмите шку...   
3  Подойти к столешнице. Найти творог. Взять твор...   
4  Подойти к холодильнику. Открыть холодильник. Н...   

                               

In [5]:
print(data['Успех предсказанного плана'].value_counts())

Успех предсказанного плана
1    3234
0    1600
Name: count, dtype: int64


In [6]:
print(data.describe())

        Unnamed: 0  Успех предсказанного плана
count  4834.000000                 4834.000000
mean   2416.500000                    0.669011
std    1395.599931                    0.470618
min       0.000000                    0.000000
25%    1208.250000                    0.000000
50%    2416.500000                    1.000000
75%    3624.750000                    1.000000
max    4833.000000                    1.000000


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Задача'])

In [9]:
print(X.shape)

(4834, 5402)


In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words and word.isalpha()]
    return ' '.join(words)

data['Задача'] = data['Задача'].apply(preprocess_text)
data['Обстановка'] = data['Обстановка'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# Инициализация векторизатора
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Объединяем текстовые данные для векторизации
combined_text = data['Задача'] + " " + data['Обстановка']
X = tfidf_vectorizer.fit_transform(combined_text)

# Целевая переменная
y = data['Успех предсказанного плана']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SMOTE

In [13]:
# Применение SMOTE для балансировки классов
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [14]:
print(X_train_resampled.shape)

(5190, 5000)


In [15]:
import numpy as np

# Посчитаем количество элементов каждого класса в y_train_resampled
unique, counts = np.unique(y_train_resampled, return_counts=True)
class_counts = dict(zip(unique, counts))

print("Баланс классов в X_train_resampled:")
print(class_counts)


Баланс классов в X_train_resampled:
{0: 2595, 1: 2595}


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Обучение классификатора
model = LogisticRegression()
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the model: {accuracy}')

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


def objective(trial):
    # Определите пространство поиска гиперпараметров
    C = trial.suggest_float("C", 0.0001, 10000)
    penalty = trial.suggest_categorical("penalty", ["l2", "none"])
    max_iter = trial.suggest_int("max_iter", 100, 1000)

    # Модель логистической регрессии
    model = LogisticRegression(
        C=C,
        penalty=penalty,
        max_iter=max_iter
    )

    # Обучение
    model.fit(X_train, y_train)

    # Предсказание
    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred)

    return f1


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

#лучшие гиперпараметры
best_params = study.best_params
best_C = best_params["C"]
best_penalty = best_params["penalty"]
best_max_iter = best_params["max_iter"]

#модель с лучшими гиперпараметрами
best_model = LogisticRegression(
    C=best_C,
    penalty=best_penalty,
    max_iter=best_max_iter
)

best_model.fit(X, y)

y_pred = best_model.predict(X_test)
#Лучшие гиперпараметры для логистической регрессии
print(best_params)

# Оценим качество
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc:.2f}")

[I 2024-04-04 21:22:31,814] A new study created in memory with name: no-name-6ea4f7b8-c798-494c-89b4-40e53f7d6507


Accuracy of the model: 0.6783867631851086
F1 Score: 0.80


C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[I 2024-04-04 21:22:32,225] Trial 0 finished with value: 0.7002360346184106 and parameters: {'C': 3818.101942355697, 'penalty': 'l2', 'max_iter': 512}. Best is trial 0 with value: 0.7002360346184106.
C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_it

# НОВОЕ

In [16]:
import numpy as np

# Посчитаем количество элементов каждого класса в y_train_resampled
unique, counts = np.unique(y_train, return_counts=True)
class_counts = dict(zip(unique, counts))

print("Баланс классов в X_train_resampled:")
print(class_counts)


Баланс классов в X_train_resampled:
{0: 1272, 1: 2595}


In [17]:
X_train = X_train_resampled

In [18]:
y_train = y_train_resampled

In [19]:
import numpy as np

# Посчитаем количество элементов каждого класса в y_train_resampled
unique, counts = np.unique(y_train, return_counts=True)
class_counts = dict(zip(unique, counts))

print("Баланс классов в X_train_resampled:")
print(class_counts)


Баланс классов в X_train_resampled:
{0: 2595, 1: 2595}


In [65]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.99


In [20]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Обучение классификатора
model = LogisticRegression()
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the model: {accuracy}')

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


def objective(trial):
    # Определите пространство поиска гиперпараметров
    C = trial.suggest_float("C", 0.0001, 10000)
    penalty = trial.suggest_categorical("penalty", ["l2", "none"])
    max_iter = trial.suggest_int("max_iter", 100, 1000)

    # Модель логистической регрессии
    model = LogisticRegression(
        C=C,
        penalty=penalty,
        max_iter=max_iter
    )

    # Обучение
    model.fit(X_train, y_train)

    # Предсказание
    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred)

    return f1


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

#лучшие гиперпараметры
best_params = study.best_params
best_C = best_params["C"]
best_penalty = best_params["penalty"]
best_max_iter = best_params["max_iter"]

#модель с лучшими гиперпараметрами
best_model = LogisticRegression(
    C=best_C,
    penalty=best_penalty,
    max_iter=best_max_iter
)

best_model.fit(X, y)

y_pred = best_model.predict(X_test)
#Лучшие гиперпараметры для логистической регрессии
print(best_params)

# Оценим качество
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc:.2f}")

C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Accuracy of the model: 0.6339193381592554
F1 Score: 0.72


[I 2024-04-04 21:25:15,212] A new study created in memory with name: no-name-ab62e569-206d-4414-b1d5-58a0ab3c379b
C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver 

{'C': 6541.303713604339, 'penalty': 'none', 'max_iter': 148}
Accuracy: 1.00
F1 Score: 1.00
ROC-AUC Score: 1.00


C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_opt

In [21]:
file_path ='final_test_data.csv'
data = pd.read_csv('final_test_data.csv')

In [22]:
# Инициализация векторизатора
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Объединяем текстовые данные для векторизации
combined_text = data['Задача en'] + " " + data['Обстановка en']
X2 = tfidf_vectorizer.fit_transform(combined_text)

# Целевая переменная
y2 = data['Успех предсказанного плана']

In [25]:

best_model.fit(X2, y2)



y_pred = best_model.predict(X2)
#Лучшие гиперпараметры для логистической регрессии
print(best_params)

# Оценим качество
accuracy = accuracy_score(y2, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(y2, y_pred)
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(y2, y_pred)
print(f"ROC-AUC Score: {roc_auc:.2f}")

C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


ValueError: Found input variables with inconsistent numbers of samples: [4834, 967]